In [1]:
import timm
import torch.nn as nn

class SwinTransformer2D(nn.Module):
    def __init__(self, num_classes):
        super(SwinTransformer2D, self).__init__()
        self.swin_transformer = timm.create_model('swin_tiny_patch4_window7_224', pretrained=True, num_classes=num_classes)

    def forward(self, x):
        x = self.swin_transformer(x)
        return x

model = SwinTransformer2D(num_classes=1000)

# 打印所有层的名称
print("所有层的名称:")
for name, module in model.named_modules():
    print(name)

# 找到并打印最后两层的名称
layer_names = [name for name, _ in model.named_modules()]
if len(layer_names) >= 2:
    print("\n最后两层的名称:")
    print(layer_names[-2:])
else:
    print("模型中没有足够的层来显示最后两层的名称")

model.safetensors:   0%|          | 0.00/114M [00:00<?, ?B/s]

所有层的名称:

swin_transformer
swin_transformer.patch_embed
swin_transformer.patch_embed.proj
swin_transformer.patch_embed.norm
swin_transformer.layers
swin_transformer.layers.0
swin_transformer.layers.0.downsample
swin_transformer.layers.0.blocks
swin_transformer.layers.0.blocks.0
swin_transformer.layers.0.blocks.0.norm1
swin_transformer.layers.0.blocks.0.attn
swin_transformer.layers.0.blocks.0.attn.qkv
swin_transformer.layers.0.blocks.0.attn.attn_drop
swin_transformer.layers.0.blocks.0.attn.proj
swin_transformer.layers.0.blocks.0.attn.proj_drop
swin_transformer.layers.0.blocks.0.attn.softmax
swin_transformer.layers.0.blocks.0.drop_path1
swin_transformer.layers.0.blocks.0.norm2
swin_transformer.layers.0.blocks.0.mlp
swin_transformer.layers.0.blocks.0.mlp.fc1
swin_transformer.layers.0.blocks.0.mlp.act
swin_transformer.layers.0.blocks.0.mlp.drop1
swin_transformer.layers.0.blocks.0.mlp.norm
swin_transformer.layers.0.blocks.0.mlp.fc2
swin_transformer.layers.0.blocks.0.mlp.drop2
swin_transforme

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import numpy as np
import os
import timm
from torchvision import transforms
from sklearn.preprocessing import LabelEncoder
from torchvision.transforms.functional import to_tensor, normalize

# 设置随机种子以确保可重复性
def set_seed(seed=42):
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

set_seed(42)

# 数据集类
class NPYDataset(Dataset):
    def __init__(self, csv_file, root_dir, transform=None):
        try:
            self.annotations = pd.read_csv(csv_file, encoding='utf-8')
        except UnicodeDecodeError:
            self.annotations = pd.read_csv(csv_file, encoding='gbk')
        self.root_dir = root_dir
        self.le = LabelEncoder()
        self.annotations['labels'] = self.annotations['labels'].apply(lambda x: x.strip("[]'"))
        self.annotations['labels'] = self.le.fit_transform(self.annotations['labels'])
        self.transform = transform

    def __len__(self):
        return len(self.annotations)

    def __getitem__(self, index):
        img_name = os.path.join(self.root_dir, str(self.annotations.iloc[index, -1]) + '.npy')
        image = np.load(img_name).astype(np.float32)  # 将数据类型转换为float32
        if self.transform:
            image = self.transform(image)
        label = self.annotations.iloc[index, 2]
        return image, label


# 定义图像转换
transform = transforms.Compose([
    transforms.ToTensor(),  # 将numpy数组转换为torch张量，并且从(H, W, C)转换为(C, H, W)且归一化到[0, 1]
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # 归一化
])

# 模型定义
class SwinTransformer2D(nn.Module):
    def __init__(self, num_classes):
        super(SwinTransformer2D, self).__init__()
        self.swin_transformer = timm.create_model('swin_tiny_patch4_window7_224', pretrained=True, num_classes=num_classes)

    def forward(self, x):
        x = self.swin_transformer(x)
        return x

# 训练和评估参数
num_epochs = 47
batch_size = 32
learning_rate = 0.001

# 设备配置
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# 数据加载
train_dataset = NPYDataset(csv_file='/kaggle/input/d-n-transformer/train.csv', root_dir='/kaggle/input/d-n-transformer/transformer_gaus_train/transformer_gaus_train', transform=transform)
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)

validation_dataset = NPYDataset(csv_file='/kaggle/input/d-n-transformer/validation.csv', root_dir='/kaggle/input/d-n-transformer/transformer_gaus_validation/transformer_gaus_validation', transform=transform)
validation_loader = DataLoader(dataset=validation_dataset, batch_size=batch_size, shuffle=False)

test_dataset = NPYDataset(csv_file='/kaggle/input/d-n-transformer/test.csv', root_dir='/kaggle/input/d-n-transformer/transformer_gaus_test/transformer_gaus_test', transform=transform)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)

# 模型初始化
num_classes = len(np.unique(train_dataset.annotations['labels']))
model = SwinTransformer2D(num_classes=num_classes).to(device)

# 创建一个权重数组，给第三类更高的权重
weights = torch.tensor([1.0, 1.0], dtype=torch.float32).to(device)

# 使用加权损失函数
criterion = nn.CrossEntropyLoss(weight=weights)

optimizer = optim.AdamW(model.parameters(), lr=0.001, weight_decay=1e-2)

# 学习率调度器
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=20, gamma=0.5)

In [3]:
# 评估函数
def evaluate_model(model, data_loader, device='cuda'):
    model.eval()  # 设置模型为评估模式
    total = 0
    correct = 0
    with torch.no_grad():
        for images, labels in data_loader:
            images = images.to(device)
            labels = labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    accuracy = 100 * correct / total
    return accuracy

def train_model(model, train_loader, validation_loader, test_loader, criterion, optimizer, scheduler, num_epochs=10, device='cuda'):
    # 解冻最后两层的参数
    for name, param in model.named_parameters():
        if 'head' in name or 'swin_transformer.norm'in name or 'swin_transformer.layers.3.blocks.1.drop_path2' in name:  # 根据层的名称来解冻
            param.requires_grad = True
        else:
            param.requires_grad = False
    
    # 确保优化器仅更新requires_grad=True的参数
    optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=0.001)
    
    for epoch in range(num_epochs):
        model.train()  # 设置模型为训练模式
        running_loss = 0.0
        
        # 训练过程
        for images, labels in train_loader:
            images = images.to(device)
            labels = labels.to(device)

            outputs = model(images)
            loss = criterion(outputs, labels)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        scheduler.step()

        # 在每个epoch结束后评估模型性能
        train_accuracy = evaluate_model(model, train_loader, device)
        validation_accuracy = evaluate_model(model, validation_loader, device)
        test_accuracy = evaluate_model(model, test_loader, device)

        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}, '
              f'Train Accuracy: {train_accuracy:.2f}%, '
              f'Validation Accuracy: {validation_accuracy:.2f}%, '
              f'Test Accuracy: {test_accuracy:.2f}%')



In [4]:
# 开始训练模型
train_model(model, train_loader, validation_loader, test_loader, criterion, optimizer, scheduler, num_epochs, device)

/opt/conda/lib/python3.10/site-packages/torch/optim/lr_scheduler.py:136: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn("Detected call of `lr_scheduler.step()` before `optimizer.step()`. "


Epoch [1/47], Loss: 0.6766, Train Accuracy: 66.71%, Validation Accuracy: 64.00%, Test Accuracy: 61.33%
Epoch [2/47], Loss: 0.6350, Train Accuracy: 59.86%, Validation Accuracy: 58.67%, Test Accuracy: 60.00%
Epoch [3/47], Loss: 0.6057, Train Accuracy: 71.57%, Validation Accuracy: 64.00%, Test Accuracy: 73.33%
Epoch [4/47], Loss: 0.5952, Train Accuracy: 71.57%, Validation Accuracy: 65.33%, Test Accuracy: 73.33%
Epoch [5/47], Loss: 0.5709, Train Accuracy: 73.43%, Validation Accuracy: 70.00%, Test Accuracy: 70.67%
Epoch [6/47], Loss: 0.5691, Train Accuracy: 73.86%, Validation Accuracy: 65.33%, Test Accuracy: 73.33%
Epoch [7/47], Loss: 0.5525, Train Accuracy: 75.29%, Validation Accuracy: 68.67%, Test Accuracy: 75.33%
Epoch [8/47], Loss: 0.5462, Train Accuracy: 74.57%, Validation Accuracy: 68.67%, Test Accuracy: 72.67%
Epoch [9/47], Loss: 0.5439, Train Accuracy: 74.57%, Validation Accuracy: 65.33%, Test Accuracy: 75.33%
Epoch [10/47], Loss: 0.5205, Train Accuracy: 73.43%, Validation Accuracy:

In [5]:
# 保存模型权重
model_path = '/kaggle/working/trained_model.pth'  # 指定模型保存路径
torch.save(model.state_dict(), model_path)  # 保存模型权重

In [6]:
# 正确的类别数，根据错误消息，应该是2
num_classes = 2

# 使用正确的类别数创建模型实例
model = SwinTransformer2D(num_classes=num_classes)

# 现在加载模型应该不会出错，因为类别数匹配
model.load_state_dict(torch.load(model_path, map_location=torch.device('cuda')))

# 根据你的需要调用 model.train() 或 model.eval()

<All keys matched successfully>

In [7]:
import torch  
import numpy as np  
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score  
  
# 假设model, test_loader, device等都已正确定义和初始化  
model = model.to(device)
model.eval()  
with torch.no_grad():  
    all_preds = []  
    all_labels = []  
    for images, labels in test_loader:  
        images, labels = images.to(device), labels.to(device).long()  
        outputs = model(images)  
        _, predicted = torch.max(outputs.data, 1)  
          
        # 收集所有预测和标签  
        all_preds.extend(predicted.view(-1).cpu().numpy())  
        all_labels.extend(labels.view(-1).cpu().numpy())  
  
    # 计算准确率  
    accuracy = 100 * np.sum(np.array(all_preds) == np.array(all_labels)) / len(all_labels)  
    print(f'Accuracy of the model on the test images: {accuracy:.2f} %')  
      
    # 计算精确率、召回率和F1分数  
    precision = precision_score(all_labels, all_preds, average='weighted')  
    recall = recall_score(all_labels, all_preds, average='weighted')  
    f1 = f1_score(all_labels, all_preds, average='weighted')  
      
    print(f'Precision: {precision:.4f}')  
    print(f'Recall: {recall:.4f}')  
    print(f'F1 Score: {f1:.4f}')  
  
    # 计算混淆矩阵  
    cm = confusion_matrix(all_labels, all_preds)  
      
    # 计算每个类别的准确率  
    class_accuracy = cm.diagonal() / cm.sum(axis=1)  
      
    # 打印每个类别的准确率  
    for i in range(len(class_accuracy)):  
        print(f'Accuracy for class {i}: {class_accuracy[i]:.2f}')  
  
# 打印所有预测结果（如果需要）  
# print('All Predictions:', all_preds)

Accuracy of the model on the test images: 80.00 %
Precision: 0.8003
Recall: 0.8000
F1 Score: 0.8000
Accuracy for class 0: 0.79
Accuracy for class 1: 0.81


In [8]:
# 打印所有预测结果
print('All Predictions:', all_preds)

All Predictions: [0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1]


In [9]:
# Assuming `le` is your LabelEncoder instance that has been fitted to the labels
print("Label mapping:")
for i, label in enumerate(train_dataset.le.classes_):
    print(f"{label}: {i}")

Label mapping:
D: 0
N: 1


In [ ]:
# 初始化一个空列表来存储预测错误的索引
error_indices = []

# 遍历所有预测和标签，比较它们是否相等
for idx, (pred, label) in enumerate(zip(all_preds, all_labels)):
    if pred != label:
        # 如果预测不等于标签，记录该实例的索引
        error_indices.append(idx)

# 输出预测错误的实例索引
print("预测错误的实例索引:", error_indices)

# 如果您想查看预测错误的详细信息，可以按以下方式打印每个错误实例的预测值和真实值
for error_index in error_indices:
    print(f'实例 {error_index} 错误预测为 {all_preds[error_index]}，实际标签为 {all_labels[error_index]}')

In [ ]:
import pandas as pd

# 给定的行号列表，已经转换为Python的索引（从0开始）
rows = [24, 34, 38, 39, 50, 57, 71, 72, 73, 86, 121, 130, 136, 141, 145, 178, 186, 193, 195, 197, 198, 200, 204, 205, 212, 220, 225, 232, 238, 242, 246, 249, 261, 264, 275, 289]
# 读取CSV文件
df = pd.read_csv('/kaggle/input/d-n-transformer/test.csv')

# 根据给定的行号提取filename列的值
selected_filenames = df.loc[rows, 'filename']

# 打印结果
print(selected_filenames)

In [ ]:
import os
import shutil
import pandas as pd

# 定义CSV文件的路径
csv_file_path = '/kaggle/input/d-n-transformer/test.csv'
# 使用Pandas读取CSV文件
try:
    df = pd.read_csv(csv_file_path)
    test_df = df
    print("CSV文件成功加载到DataFrame中。")
except FileNotFoundError:
    print(f"未找到文件：{csv_file_path}。请检查文件路径是否正确。")
except Exception as e:
    print(f"读取CSV文件时出错：{e}")

# 原始图像文件夹路径
source_folder = '/kaggle/input/transformer-gaus-images/normalized_gaus_test/normalized_gaus_test'

# 新的测试集图像文件夹路径
test_images_folder = '/kaggle/working/all_normalized_double_test_images'

# 创建新的文件夹（如果不存在）
os.makedirs(test_images_folder, exist_ok=True)

# 从DataFrame中提取测试集的图像文件名
test_filenames = test_df['filename'].tolist()

# 复制测试集图像到新的测试集文件夹
for filename in test_filenames:
    # 确保文件名是字符串类型，添加.npy扩展名（如果需要）
    filename_str = str(filename) + '.npy'  # 假设你需要添加.npy扩展名
    source_path = os.path.join(source_folder, filename_str)
    destination_path = os.path.join(test_images_folder, filename_str)
    # 检查源文件是否存在，再复制
    if os.path.exists(source_path):
        shutil.copy(source_path, destination_path)
    else:
        print(f"文件不存在：{source_path}")

print("图像已复制到相应的测试集文件夹。")

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix

# 基于之前的预测结果计算混淆矩阵
cm = confusion_matrix(all_labels, all_preds)

# 绘制混淆矩阵的热力图
plt.figure(figsize=(12, 10))  # 根据需要调整图像的大小
sns.heatmap(cm, annot=True, fmt="d", cmap="YlGnBu", xticklabels=range(len(np.unique(all_labels))), yticklabels=range(len(np.unique(all_labels))))
plt.xlabel('Predicted labels')
plt.ylabel('True labels')
plt.title('Confusion Matrix')
plt.show()

In [ ]:
# 保存模型权重
model_path = '/kaggle/working/trained_model.pth'  # 指定模型保存路径
torch.save(model.state_dict(), model_path)  # 保存模型权重